In [1]:
#Importamos las librerias

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import kerastuner as kt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop

In [3]:
#Leemos los datos

In [4]:
df=pd.read_csv('df_regresion_1400.csv')
df

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,-7.621282,-3.644223,4.479734,-5.269715,2.448071,0.481839,5.534079,-5.268483,-51.083965
1,-2.397800,-1.152185,-4.598190,-4.509740,1.578773,6.880768,-2.044546,0.858393,-16.430133
2,-0.126112,9.818202,-1.402474,-5.637452,-3.480554,-1.998603,4.222707,6.671159,-199.186533
3,2.819058,-9.421755,0.699978,-6.613341,-2.738968,-1.452410,-0.934815,-2.491929,-172.747680
4,1.011369,0.410828,4.315859,-0.677440,5.740309,-1.643752,-6.652297,-4.903230,11.633725
...,...,...,...,...,...,...,...,...,...
1395,0.288420,0.867375,-13.057251,8.144130,2.620245,1.224767,-4.083578,-0.877677,-41.607722
1396,-7.141025,-1.159053,-2.452662,-4.567519,0.463623,-4.275232,-6.837313,5.195260,-42.043155
1397,-9.271913,-9.306076,12.288837,1.546173,9.789573,4.275759,7.225917,-6.437789,-167.247791
1398,-9.281869,-7.097289,7.470348,-1.714904,-1.783365,-2.126855,-3.032398,-6.251700,-160.403773


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      1400 non-null   float64
 1   x2      1400 non-null   float64
 2   x3      1400 non-null   float64
 3   x4      1400 non-null   float64
 4   x5      1400 non-null   float64
 5   x6      1400 non-null   float64
 6   x7      1400 non-null   float64
 7   x8      1400 non-null   float64
 8   y       1400 non-null   float64
dtypes: float64(9)
memory usage: 98.6 KB


In [5]:
df.describe()

,x1,x2,x3,x4,x5,x6,x7,x8,y
count,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000,1400.000000
mean,0.051127,-0.013543,-0.151060,0.042390,0.068184,0.134942,0.147108,-0.078500,-43.358291
std,4.984275,5.005276,5.031547,4.884986,4.947941,4.982442,5.084304,5.180180,126.330271
min,-16.291303,-15.944120,-16.199604,-17.053131,-16.800671,-17.674901,-16.302739,-15.537009,-2367.430110
25%,-3.402535,-3.473621,-3.708008,-3.264911,-3.144907,-3.227951,-3.124883,-3.651020,-71.696215
50%,0.077756,-0.218939,-0.089413,-0.032082,0.219778,0.094248,-0.010826,-0.110169,-27.829004
75%,3.356497,3.430782,3.383758,3.300334,3.327204,3.427676,3.608301,3.522290,-0.591042
max,19.110318,14.604678,16.457801,13.742084,15.113273,18.564119,19.814182,22.214982,1636.243361


In [6]:
Q1 = df['y'].quantile(0.25)
print("primer cuartil",Q1)
Q3 = df['y'].quantile(0.75)
print('Tercer cuartil',Q3)
IQR=Q3-Q1
print('Rango Intercuartil',IQR)

primer cuartil -71.69621542421173
Tercer cuartil -0.5910419258222894
Rango Intercuartil 71.10517349838943


In [7]:
inf=(Q1-1.5*IQR)
print('inf',inf)
sup=(Q3+1.5*IQR)
print('sup',sup)

inf -178.35397567179587
sup 106.06671832176185


In [8]:
ubicacion=(df['y']<inf) | (df['y']>sup)
print('ubicacion de outliers',ubicacion)

ubicacion de outliers 0       False
1       False
2        True
3       False
4       False
        ...  
1395    False
1396    False
1397    False
1398    False
1399    False
Name: y, Length: 1400, dtype: bool


In [9]:
Outliers = df[ubicacion]
print('lista outliers',Outliers)

lista outliers              x1         x2        x3        x4         x5         x6  \
2     -0.126112   9.818202 -1.402474 -5.637452  -3.480554  -1.998603   
8     10.290715  -0.894308  3.253156  5.845159   3.112735   2.382046   
9     -0.081090   9.567365 -3.779233  7.475284  -1.744231  -0.705118   
15   -10.596003   6.640003 -8.324260  0.847868  -5.708803  -1.413653   
20    -1.774709   9.431061 -1.790464  1.666379  -2.818530   8.761653   
...         ...        ...       ...       ...        ...        ...   
1348  -3.113640 -11.254869  1.696785 -3.852455   6.882917 -13.125212   
1350  -2.576504   0.089292  9.491170  3.806541  11.524729  -0.019419   
1355   2.560743  -9.688525  1.827161 -0.979038  -5.084225  -1.136919   
1378   5.616213 -10.961910 -7.151388  5.624010   8.257645   2.687449   
1386  12.133571  -3.688319  0.478167  3.874232  -1.930445  -2.647407   

             x7        x8           y  
2      4.222707  6.671159 -199.186533  
8      2.589481  4.032536  115.707325  


In [10]:
ubicacion_sinoutliers=(df['y'] >= inf) & (df['y'] <= sup)
dflimpio=df[ubicacion_sinoutliers]
dflimpio

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,-7.621282,-3.644223,4.479734,-5.269715,2.448071,0.481839,5.534079,-5.268483,-51.083965
1,-2.397800,-1.152185,-4.598190,-4.509740,1.578773,6.880768,-2.044546,0.858393,-16.430133
3,2.819058,-9.421755,0.699978,-6.613341,-2.738968,-1.452410,-0.934815,-2.491929,-172.747680
4,1.011369,0.410828,4.315859,-0.677440,5.740309,-1.643752,-6.652297,-4.903230,11.633725
5,1.812883,4.592557,-3.936672,-2.809403,2.578789,-2.567569,7.568280,-11.072693,-44.572481
...,...,...,...,...,...,...,...,...,...
1395,0.288420,0.867375,-13.057251,8.144130,2.620245,1.224767,-4.083578,-0.877677,-41.607722
1396,-7.141025,-1.159053,-2.452662,-4.567519,0.463623,-4.275232,-6.837313,5.195260,-42.043155
1397,-9.271913,-9.306076,12.288837,1.546173,9.789573,4.275759,7.225917,-6.437789,-167.247791
1398,-9.281869,-7.097289,7.470348,-1.714904,-1.783365,-2.126855,-3.032398,-6.251700,-160.403773


In [11]:
#division train y test
X= dflimpio.drop(axis=1, columns= 'y')
y= dflimpio['y']

# Normalizamos
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))  



# Separamos en Train y Test Sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.20, random_state= 123)
#X_val_train, X_val_test, y_val_train, y_val_test = train_test_split(X_train, y_train, test_size= 0.20, random_state= 123)

#lo entrenamos con el train

scaler.fit(X_train)


    #lo aplicamos al Train y al Test

X_train_n=scaler.transform(X_train)
X_test_n=scaler.transform(X_test)

In [12]:
#se crea el modelo 

In [13]:
def crear_modelo(hp): 
    
    # arquitectura:
    modelo = Sequential()
    
    #cargamos la capa de entrada
    modelo.add(Input(shape=(8,), name= "Entrada"))
    
    # cargamos los valores de los hiperparámetros
    
    hp_initializer=hp.Choice('kernel_initializer',values=['uniform','glorot_uniform',"Zeros"])
    
    hp_activation= hp.Choice( 'activacion', values=['relu','selu'])
    
    hp_optimizer=hp.Choice('optimizer', values=['sgd','adam'])
    
    hp_neuronas_1=hp.Int(name='units_1',min_value=1,max_value=30, step=4)
    
    hp_neuronas_2=hp.Int(name='units_2',min_value=1,max_value=30, step=2)
    
    hp_neuronas_3=hp.Int(name='units_3',min_value=1,max_value=30, step=2)
    
    # Ahora vamos con las hidden layers
    modelo.add(Dense(units=hp_neuronas_1, kernel_initializer= hp_initializer, activation= hp_activation ))
    
    modelo.add(Dense(units=hp_neuronas_2, kernel_initializer= hp_initializer ,  activation= hp_activation ))
    
    modelo.add(Dense(units=hp_neuronas_3, kernel_initializer= hp_initializer ,  activation= hp_activation ))
    
    # Capa de salida: observe que en la capa de salida pasamos el valor fijo de 'sigmoid' para activacion
    modelo.add(Dense(units=1,activation ='linear' )) #, kernel_initializer= hp_initializer , activation= ''
    
    
    # compilación:
    modelo.compile(loss= 'mean_squared_error' , optimizer= hp_optimizer , metrics=[ 'mean_squared_error' ])
    
    return modelo

In [14]:
tuner=kt.Hyperband(crear_modelo,
                     objective = 'mean_squared_error', 
                     max_epochs = 100,
                     factor = 3,
                     directory = 'C:\salida-hyperband', #Le indico el directorio en la raíz de otra unidad, si no me da error de permisos
                     project_name = 'Iefi01xx')  # OJO CAMBIAR NOMBRE DE ARCHIVO EN CADA CORRIDA


In [15]:
import IPython
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [16]:
tuner.search(X_train_n, y_train, epochs = 100, 
             validation_split=0.2)

Epoch 1/2
26/26 [==============================] - ETA: 1:18 - loss: 2886.1155 - mean_squared_error: 2886.11 - ETA: 0s - loss: 3366.2832 - mean_squared_error: 3366.2832 - ETA: 0s - loss: 2905.6245 - mean_squared_error: 2905.62 - ETA: 0s - loss: 3041.4524 - mean_squared_error: 3041.45 - ETA: 0s - loss: 2897.4478 - mean_squared_error: 2897.44 - ETA: 0s - loss: 2896.8794 - mean_squared_error: 2896.87 - ETA: 0s - loss: 2805.7214 - mean_squared_error: 2805.72 - ETA: 0s - loss: 2780.5103 - mean_squared_error: 2780.51 - 4s 36ms/step - loss: 2780.5103 - mean_squared_error: 2780.5103 - val_loss: 2658.4685 - val_mean_squared_error: 2658.4685
Epoch 2/2
26/26 [==============================] - ETA: 0s - loss: 2438.9722 - mean_squared_error: 2438.97 - ETA: 0s - loss: 2356.9041 - mean_squared_error: 2356.90 - ETA: 0s - loss: 2640.4072 - mean_squared_error: 2640.40 - ETA: 0s - loss: 2513.2844 - mean_squared_error: 2513.28 - ETA: 0s - loss: 2607.5771 - mean_squared_error: 2607.57 - ETA: 0s - loss: 268

Epoch 1/2
26/26 [==============================] - ETA: 23s - loss: 2725.4224 - mean_squared_error: 2725.422 - ETA: 0s - loss: 2872.7607 - mean_squared_error: 2872.760 - ETA: 0s - loss: 3018.3765 - mean_squared_error: 3018.37 - ETA: 0s - loss: 2883.9939 - mean_squared_error: 2883.99 - ETA: 0s - loss: 2921.0034 - mean_squared_error: 2921.00 - ETA: 0s - loss: 2857.0884 - mean_squared_error: 2857.08 - 2s 28ms/step - loss: 2802.8945 - mean_squared_error: 2802.8945 - val_loss: 3182.5776 - val_mean_squared_error: 3182.5776
Epoch 2/2
26/26 [==============================] - ETA: 0s - loss: 2914.3423 - mean_squared_error: 2914.34 - ETA: 0s - loss: 2573.5854 - mean_squared_error: 2573.58 - ETA: 0s - loss: 2730.0005 - mean_squared_error: 2730.00 - ETA: 0s - loss: 2852.6414 - mean_squared_error: 2852.64 - ETA: 0s - loss: 2926.9766 - mean_squared_error: 2926.97 - ETA: 0s - loss: 2789.4307 - mean_squared_error: 2789.43 - 0s 15ms/step - loss: 2781.0876 - mean_squared_error: 2781.0876 - val_loss: 262

Epoch 1/2
26/26 [==============================] - ETA: 25s - loss: 4283.0283 - mean_squared_error: 4283.028 - ETA: 0s - loss: 3567.1760 - mean_squared_error: 3567.176 - ETA: 0s - loss: 3801.4006 - mean_squared_error: 3801.40 - ETA: 0s - loss: 3602.8281 - mean_squared_error: 3602.82 - 1s 17ms/step - loss: 3606.8765 - mean_squared_error: 3606.8765 - val_loss: 3493.1038 - val_mean_squared_error: 3493.1038
Epoch 2/2
26/26 [==============================] - ETA: 0s - loss: 2025.2070 - mean_squared_error: 2025.20 - ETA: 0s - loss: 3639.3359 - mean_squared_error: 3639.33 - ETA: 0s - loss: 3788.4299 - mean_squared_error: 3788.42 - ETA: 0s - loss: 3429.9036 - mean_squared_error: 3429.90 - ETA: 0s - loss: 3392.8770 - mean_squared_error: 3392.87 - 0s 12ms/step - loss: 3463.9905 - mean_squared_error: 3463.9905 - val_loss: 3309.1448 - val_mean_squared_error: 3309.1448


Epoch 1/2
26/26 [==============================] - ETA: 18s - loss: 2914.3096 - mean_squared_error: 2914.309 - ETA: 0s - loss: 2853.1428 - mean_squared_error: 2853.142 - ETA: 0s - loss: 3273.0867 - mean_squared_error: 3273.08 - ETA: 0s - loss: 3513.4263 - mean_squared_error: 3513.42 - ETA: 0s - loss: 3603.1399 - mean_squared_error: 3603.13 - 1s 21ms/step - loss: 3644.8181 - mean_squared_error: 3644.8181 - val_loss: 3586.1885 - val_mean_squared_error: 3586.1885
Epoch 2/2
26/26 [==============================] - ETA: 0s - loss: 3175.9199 - mean_squared_error: 3175.91 - ETA: 0s - loss: 3699.0457 - mean_squared_error: 3699.04 - ETA: 0s - loss: 3744.8579 - mean_squared_error: 3744.85 - ETA: 0s - loss: 3972.7256 - mean_squared_error: 3972.72 - ETA: 0s - loss: 3680.6169 - mean_squared_error: 3680.61 - 0s 12ms/step - loss: 3616.6790 - mean_squared_error: 3616.6790 - val_loss: 3548.7214 - val_mean_squared_error: 3548.7214


Epoch 1/2
26/26 [==============================] - ETA: 21s - loss: 5913.2461 - mean_squared_error: 5913.246 - ETA: 0s - loss: 4090.4004 - mean_squared_error: 4090.400 - ETA: 0s - loss: 4297.9399 - mean_squared_error: 4297.93 - ETA: 0s - loss: 3737.1846 - mean_squared_error: 3737.18 - ETA: 0s - loss: 3651.8145 - mean_squared_error: 3651.81 - 1s 20ms/step - loss: 3653.3496 - mean_squared_error: 3653.3496 - val_loss: 3605.0393 - val_mean_squared_error: 3605.0393
Epoch 2/2
26/26 [==============================] - ETA: 0s - loss: 3246.4346 - mean_squared_error: 3246.43 - ETA: 0s - loss: 3974.9436 - mean_squared_error: 3974.94 - ETA: 0s - loss: 3970.8687 - mean_squared_error: 3970.86 - ETA: 0s - loss: 3821.2554 - mean_squared_error: 3821.25 - ETA: 0s - loss: 3722.1067 - mean_squared_error: 3722.10 - 0s 12ms/step - loss: 3644.7305 - mean_squared_error: 3644.7305 - val_loss: 3596.0635 - val_mean_squared_error: 3596.0635


...

26/26 [==============================] - ETA: 0s - loss: 4360.4746 - mean_squared_error: 4360.47 - ETA: 0s - loss: 2680.0127 - mean_squared_error: 2680.01 - ETA: 0s - loss: 2709.2822 - mean_squared_error: 2709.28 - ETA: 0s - loss: 2692.1243 - mean_squared_error: 2692.12 - ETA: 0s - loss: 2706.7278 - mean_squared_error: 2706.72 - 0s 14ms/step - loss: 2751.4077 - mean_squared_error: 2751.4077 - val_loss: 2758.3335 - val_mean_squared_error: 2758.3335
Epoch 87/100
26/26 [==============================] - ETA: 0s - loss: 2708.8049 - mean_squared_error: 2708.80 - ETA: 0s - loss: 2690.4182 - mean_squared_error: 2690.41 - ETA: 0s - loss: 2543.3281 - mean_squared_error: 2543.32 - ETA: 0s - loss: 2610.5542 - mean_squared_error: 2610.55 - ETA: 0s - loss: 2756.9487 - mean_squared_error: 2756.94 - 0s 13ms/step - loss: 2745.5164 - mean_squared_error: 2745.5164 - val_loss: 2753.3967 - val_mean_squared_error: 2753.3967
Epoch 88/100
26/26 [==============================] - ETA: 0s - loss: 3154.6279 - m

Epoch 1/100
26/26 [==============================] - ETA: 27s - loss: 2339.6572 - mean_squared_error: 2339.657 - ETA: 0s - loss: 3469.6592 - mean_squared_error: 3469.659 - ETA: 0s - loss: 3820.5469 - mean_squared_error: 3820.54 - ETA: 0s - loss: 3917.2551 - mean_squared_error: 3917.25 - ETA: 0s - loss: 3689.9539 - mean_squared_error: 3689.95 - 2s 22ms/step - loss: 3654.3872 - mean_squared_error: 3654.3872 - val_loss: 3607.2603 - val_mean_squared_error: 3607.2603
Epoch 2/100
26/26 [==============================] - ETA: 0s - loss: 4920.9229 - mean_squared_error: 4920.92 - ETA: 0s - loss: 4325.9951 - mean_squared_error: 4325.99 - ETA: 0s - loss: 3657.1250 - mean_squared_error: 3657.12 - ETA: 0s - loss: 3682.0818 - mean_squared_error: 3682.08 - ETA: 0s - loss: 3686.5852 - mean_squared_error: 3686.58 - 0s 13ms/step - loss: 3648.5149 - mean_squared_error: 3648.5149 - val_loss: 3601.0325 - val_mean_squared_error: 3601.0325
Epoch 3/100
26/26 [==============================] - ETA: 0s - loss: 

26/26 [==============================] - ETA: 0s - loss: 3303.5020 - mean_squared_error: 3303.50 - ETA: 0s - loss: 3506.2329 - mean_squared_error: 3506.23 - ETA: 0s - loss: 3075.5186 - mean_squared_error: 3075.51 - ETA: 0s - loss: 3042.2371 - mean_squared_error: 3042.23 - ETA: 0s - loss: 2945.2471 - mean_squared_error: 2945.24 - ETA: 0s - loss: 3045.2000 - mean_squared_error: 3045.20 - 0s 15ms/step - loss: 3036.3562 - mean_squared_error: 3036.3562 - val_loss: 3012.6101 - val_mean_squared_error: 3012.6101
Epoch 34/100
26/26 [==============================] - ETA: 0s - loss: 4283.7251 - mean_squared_error: 4283.72 - ETA: 0s - loss: 3185.2930 - mean_squared_error: 3185.29 - ETA: 0s - loss: 2772.3965 - mean_squared_error: 2772.39 - ETA: 0s - loss: 2763.0608 - mean_squared_error: 2763.06 - ETA: 0s - loss: 2920.5352 - mean_squared_error: 2920.53 - 0s 13ms/step - loss: 3014.1624 - mean_squared_error: 3014.1624 - val_loss: 2992.6084 - val_mean_squared_error: 2992.6084
Epoch 35/100
26/26 [=====

26/26 [==============================] - ETA: 0s - loss: 1956.3806 - mean_squared_error: 1956.38 - ETA: 0s - loss: 2375.8860 - mean_squared_error: 2375.88 - ETA: 0s - loss: 2732.6685 - mean_squared_error: 2732.66 - ETA: 0s - loss: 2513.6697 - mean_squared_error: 2513.66 - ETA: 0s - loss: 2599.5691 - mean_squared_error: 2599.56 - ETA: 0s - loss: 2616.7625 - mean_squared_error: 2616.76 - 0s 16ms/step - loss: 2616.7625 - mean_squared_error: 2616.7625 - val_loss: 2644.8701 - val_mean_squared_error: 2644.8701
Epoch 68/100
26/26 [==============================] - ETA: 0s - loss: 1773.7682 - mean_squared_error: 1773.76 - ETA: 0s - loss: 2424.4978 - mean_squared_error: 2424.49 - ETA: 0s - loss: 2356.4951 - mean_squared_error: 2356.49 - ETA: 0s - loss: 2602.7664 - mean_squared_error: 2602.76 - ETA: 0s - loss: 2638.6782 - mean_squared_error: 2638.67 - ETA: 0s - loss: 2608.8596 - mean_squared_error: 2608.85 - 0s 15ms/step - loss: 2613.6316 - mean_squared_error: 2613.6316 - val_loss: 2642.5439 - v

INFO:tensorflow:Oracle triggered exit


In [17]:
mejores_hiperparametros = tuner.get_best_hyperparameters(num_trials = 1)[0]


In [18]:
mejores_hiperparametros.get('kernel_initializer')

'glorot_uniform'

In [19]:
mejores_hiperparametros.get('activacion')

'relu'

In [20]:
mejores_hiperparametros.get('optimizer')  

'sgd'

In [21]:
mejores_hiperparametros.get('units_1')

17

In [22]:
mejores_hiperparametros.get('units_2')

7

In [23]:
mejores_hiperparametros.get('units_3')

17

In [24]:
#Entrenamos el modelo con los mejores parametros 

In [25]:
modelo = tuner.hypermodel.build(mejores_hiperparametros)
modelo.fit(X_train_n, y_train, epochs = 100, 
             validation_data = (X_test_n, y_test))

Epoch 1/100
32/32 [==============================] - ETA: 24s - loss: 3897.3279 - mean_squared_error: 3897.327 - ETA: 0s - loss: 4962.2764 - mean_squared_error: 4962.276 - ETA: 0s - loss: 4395.6729 - mean_squared_error: 4395.67 - ETA: 0s - loss: 3865.3105 - mean_squared_error: 3865.31 - ETA: 0s - loss: 3513.4053 - mean_squared_error: 3513.40 - ETA: 0s - loss: 3312.2910 - mean_squared_error: 3312.29 - 1s 19ms/step - loss: 3217.4961 - mean_squared_error: 3217.4961 - val_loss: 2559.5745 - val_mean_squared_error: 2559.5745
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 2052.5183 - mean_squared_error: 2052.51 - ETA: 0s - loss: 2689.7312 - mean_squared_error: 2689.73 - ETA: 0s - loss: 2749.8801 - mean_squared_error: 2749.88 - ETA: 0s - loss: 2672.6956 - mean_squared_error: 2672.69 - ETA: 0s - loss: 2706.9077 - mean_squared_error: 2706.90 - 0s 10ms/step - loss: 2746.1494 - mean_squared_error: 2746.1494 - val_loss: 2326.9348 - val_mean_squared_error: 2326.9348
Epoch 3/100

32/32 [==============================] - ETA: 0s - loss: 3075.8945 - mean_squared_error: 3075.89 - ETA: 0s - loss: 2679.2463 - mean_squared_error: 2679.24 - ETA: 0s - loss: 2472.1301 - mean_squared_error: 2472.13 - ETA: 0s - loss: 2524.1206 - mean_squared_error: 2524.12 - ETA: 0s - loss: 2556.1267 - mean_squared_error: 2556.12 - ETA: 0s - loss: 2600.4333 - mean_squared_error: 2600.43 - 0s 11ms/step - loss: 2600.4333 - mean_squared_error: 2600.4333 - val_loss: 2186.8735 - val_mean_squared_error: 2186.8735
Epoch 32/100
32/32 [==============================] - ETA: 0s - loss: 2848.1292 - mean_squared_error: 2848.12 - ETA: 0s - loss: 2349.4565 - mean_squared_error: 2349.45 - ETA: 0s - loss: 2741.6045 - mean_squared_error: 2741.60 - ETA: 0s - loss: 2794.0950 - mean_squared_error: 2794.09 - ETA: 0s - loss: 2654.4299 - mean_squared_error: 2654.42 - ETA: 0s - loss: 2588.5474 - mean_squared_error: 2588.54 - 0s 11ms/step - loss: 2588.5474 - mean_squared_error: 2588.5474 - val_loss: 2189.6106 - v

32/32 [==============================] - ETA: 0s - loss: 2734.3726 - mean_squared_error: 2734.37 - ETA: 0s - loss: 2559.8328 - mean_squared_error: 2559.83 - ETA: 0s - loss: 2479.7310 - mean_squared_error: 2479.73 - ETA: 0s - loss: 2561.3440 - mean_squared_error: 2561.34 - ETA: 0s - loss: 2622.5835 - mean_squared_error: 2622.58 - ETA: 0s - loss: 2589.1479 - mean_squared_error: 2589.14 - 0s 11ms/step - loss: 2594.4573 - mean_squared_error: 2594.4573 - val_loss: 2186.3481 - val_mean_squared_error: 2186.3481
Epoch 61/100
32/32 [==============================] - ETA: 0s - loss: 2168.7231 - mean_squared_error: 2168.72 - ETA: 0s - loss: 2540.5374 - mean_squared_error: 2540.53 - ETA: 0s - loss: 2466.1794 - mean_squared_error: 2466.17 - ETA: 0s - loss: 2632.3254 - mean_squared_error: 2632.32 - ETA: 0s - loss: 2613.2764 - mean_squared_error: 2613.27 - ETA: 0s - loss: 2594.4531 - mean_squared_error: 2594.45 - 0s 11ms/step - loss: 2594.4531 - mean_squared_error: 2594.4531 - val_loss: 2186.5613 - v

Epoch 89/100
32/32 [==============================] - ETA: 0s - loss: 1806.1804 - mean_squared_error: 1806.18 - ETA: 0s - loss: 2562.6797 - mean_squared_error: 2562.67 - ETA: 0s - loss: 2585.6487 - mean_squared_error: 2585.64 - ETA: 0s - loss: 2604.6294 - mean_squared_error: 2604.62 - ETA: 0s - loss: 2655.5974 - mean_squared_error: 2655.59 - ETA: 0s - loss: 2609.0510 - mean_squared_error: 2609.05 - 0s 12ms/step - loss: 2594.4746 - mean_squared_error: 2594.4746 - val_loss: 2186.3479 - val_mean_squared_error: 2186.3479
Epoch 90/100
32/32 [==============================] - ETA: 0s - loss: 2806.5178 - mean_squared_error: 2806.51 - ETA: 0s - loss: 2770.6450 - mean_squared_error: 2770.64 - ETA: 0s - loss: 2658.0574 - mean_squared_error: 2658.05 - ETA: 0s - loss: 2541.9121 - mean_squared_error: 2541.91 - ETA: 0s - loss: 2678.6311 - mean_squared_error: 2678.63 - ETA: 0s - loss: 2656.9041 - mean_squared_error: 2656.90 - 0s 12ms/step - loss: 2593.7358 - mean_squared_error: 2593.7358 - val_loss: 

In [26]:
df1=pd.read_csv('df_regresion_100_pronosticar.csv')
df1

,x1,x2,x3,x4,x5,x6,x7,x8
0,-1.512258,-0.007956,-2.155754,-4.873596,-1.390800,-15.666328,3.211058,-3.380994
1,-4.162217,3.214374,-2.061848,4.682960,-1.662741,7.037488,4.943300,5.271458
2,0.461022,-5.656724,-3.209743,-3.369872,3.496101,-7.047717,-1.891700,-5.770204
3,-0.840854,-2.943714,-11.050602,2.107808,11.469047,-5.721717,-8.337706,-4.411708
4,2.367258,-0.847152,-0.171629,-5.202584,-6.568315,0.079459,1.452082,-6.509899
...,...,...,...,...,...,...,...,...
95,-3.183691,3.503019,8.117199,-0.347410,-1.153256,-6.618298,-5.617143,0.191413
96,1.897172,-0.625657,0.627030,7.779793,7.427328,-4.410891,-8.304931,5.828320
97,4.326889,0.758917,4.402156,0.372399,-4.846391,-9.413098,1.308526,4.752488
98,-2.919018,8.740006,-3.724084,0.399934,-3.310653,-0.000224,8.049960,-0.664381


In [27]:
df1_n=scaler.transform(df1)

In [28]:
prediccion=modelo.predict(df1_n)
prediccion

array([[-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],


In [29]:
 prediccion1=modelo.predict(df1)
prediccion1

array([[-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],
       [-32.88734],


In [41]:
from sklearn.model_selection import cross_val_score

In [49]:
scores = cross_val_score(modelo,X_train_n,y_train,scoring='accuracy',cv=5)
scores

TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x000002C499F780A0>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [50]:
pd.DataFrame(prediccion).to_csv('Gonzalo_Matos_Montoya_Prediccion.csv', index=False)

In [51]:
df3=pd.read_csv('Gonzalo_Matos_Montoya_Prediccion.csv')
df3

,0
0,-32.88734
1,-32.88734
2,-32.88734
3,-32.88734
4,-32.88734
...,...
95,-32.88734
96,-32.88734
97,-32.88734
98,-32.88734
